In [55]:
import numpy as np
import IPython
from IPython.display import Audio
import torch
import torch.nn as nn
import tqdm
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [56]:
# from data import AudioData, get_dataloaders
from data2 import AudioData, get_dataloaders

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [58]:
n_mfcc = 40
fs = 22050
ad = AudioData(n_mfcc = n_mfcc)
def display(data, fs=fs):
    IPython.display.display(Audio(data, rate=fs))

In [49]:
train_dataloader, test_dataloader = get_dataloaders(ad)

In [50]:
print(ad.sampling_frequency)

22050


In [51]:
for e in train_dataloader:
    X,y, aux = e
    print(X.shape)
    print(y.shape)
    print(aux[0][0].shape)
    print('decoding x')
    decoded_X = ad.decode(aux[0])
    print('decoded, displaying...')
    display(decoded_X)
    print('displayed')
    print(X.shape)
    print(y.shape)
    print('decoding y')
    decoded_y = ad.decode(aux[1])
    print('decoded, dispalying')
    display(decoded_y)
    print('displayyed')
    break

torch.Size([1, 2031, 41])
torch.Size([1, 2031, 41])
torch.Size([1, 1463, 41])
decoding x
decoded, displaying...


displayed
torch.Size([1, 2031, 41])
torch.Size([1, 2031, 41])
decoding y
decoded, dispalying


displayyed


In [52]:
class MLP(nn.Module):
    def __init__(self, n_mfcc=40):
        super(MLP, self).__init__()
        self.l1 = torch.nn.Linear(n_mfcc, 128)
        self.l2 = torch.nn.Linear(128, n_mfcc)
        
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x)
        return x

In [53]:
linear = MLP(n_mfcc=40).to(device)

In [54]:
loss = nn.MSELoss()
optimizer = optim.Adam(linear.parameters())

EPOCHS = 1

losses = []
validation_loss = []

for i in range(EPOCHS):
    t = tqdm.tqdm(train_dataloader)
    for batch in t:
        linear.zero_grad()
        batchX, batchY, aux = batch
        batchX = torch.squeeze(batchX)
        batchY = torch.squeeze(batchY)

        batchX = batchX.type(torch.FloatTensor)
        batchY = batchY.type(torch.FloatTensor)

        batchX = batchX.to(device)
        batchY = batchY.to(device)
        
        out = linear(batchX)
        l = loss(out, batchY)
        l.backward()
        losses.append(l)
        optimizer.step()
        
        t.postfix = {'LOSS':float(l)}

  0%|          | 0/95 [00:03<?, ?it/s]


RuntimeError: mat1 dim 1 must match mat2 dim 0

In [ ]:
for e in train_dataloader:
    batchX, batchY, aux = batch
    batchX = aux[0][0]
    batchX = torch.squeeze(batchX)
    batchY = torch.squeeze(batchY)

    batchX = batchX.type(torch.FloatTensor)
    batchY = batchY.type(torch.FloatTensor)

    batchX = batchX.to(device)
    batchY = batchY.to(device)

    out = linear(batchX)
    
    pred = aux[0]
    print(pred[0].shape)
    break

In [ ]:
pred[0][0,:,1:].shape

In [ ]:
aux[0][0].shape, out.shape